# Import Statements

In [1]:
import yfinance as yf
import plotly.graph_objects as go
import pandas as pd
from datetime import datetime
import pandas_market_calendars as mcal

In [2]:
sofi = yf.Ticker("SOFI")
sofi.info

{'address1': '234 1st Street',
 'city': 'San Francisco',
 'state': 'CA',
 'zip': '94105',
 'country': 'United States',
 'phone': '855 456 7634',
 'website': 'https://www.sofi.com',
 'industry': 'Credit Services',
 'industryKey': 'credit-services',
 'industryDisp': 'Credit Services',
 'sector': 'Financial Services',
 'sectorKey': 'financial-services',
 'sectorDisp': 'Financial Services',
 'longBusinessSummary': "SoFi Technologies, Inc. provides various financial services in the United States, Latin America, Canada, and Hong Kong. It operates through three segments: Lending, Technology Platform, and Financial Services. The company offers lending and financial services and products that allows its members to borrow, save, spend, invest, and protect money; and personal loans, student loans, home loans, and related services. The company also operates Galileo, a technology platform that offers services to financial and non-financial institution; and Technisys, a cloud-native digital and core

In [7]:
hist = sofi.history(period="1d", interval="1m")

"""
# Example: NYSE calendar (SOFI)
nyse = mcal.get_calendar("NYSE")

# Get all valid trading days over your data range
schedule = nyse.schedule(start_date=hist.index.min().date(),
                         end_date=hist.index.max().date())
trading_days = schedule.index  # DatetimeIndex of trading days
mask = hist.index.tz_convert(None).normalize().isin(trading_days)
print(trading_days)
print(mask)
# Keep only rows whose date is a trading day
hist = hist[mask]
"""
hist = hist.reset_index()
hist

,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2025-12-02 09:30:00-05:00,29.389999,29.520000,29.320000,29.330000,987645,0.0,0.0
1,2025-12-02 09:31:00-05:00,29.330000,29.528200,29.315001,29.497999,221361,0.0,0.0
2,2025-12-02 09:32:00-05:00,29.500000,29.629999,29.480101,29.629999,186786,0.0,0.0
3,2025-12-02 09:33:00-05:00,29.629999,29.700001,29.566601,29.650000,245665,0.0,0.0
4,2025-12-02 09:34:00-05:00,29.654100,29.806000,29.540001,29.750000,336649,0.0,0.0
...,...,...,...,...,...,...,...,...
385,2025-12-02 15:55:00-05:00,29.610001,29.613001,29.500000,29.502001,204383,0.0,0.0
386,2025-12-02 15:56:00-05:00,29.504999,29.530001,29.500000,29.510000,176829,0.0,0.0
387,2025-12-02 15:57:00-05:00,29.510000,29.510000,29.450001,29.465000,390465,0.0,0.0
388,2025-12-02 15:58:00-05:00,29.469999,29.500000,29.459999,29.500000,217174,0.0,0.0


In [9]:
fig = go.Figure(data=[go.Candlestick(x=hist['Datetime'],
                open=hist['Open'],
                high=hist['High'],
                low=hist['Low'],
                close=hist['Close'])])

fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]),                # hide weekends
        dict(bounds=[16, 9.5], pattern="hour"),     # hide 4pm–9:30am

    ]
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    height=800
)
fig.show()